<a href="https://colab.research.google.com/github/YuriFarias741/A.I-Studies/blob/master/GoogleMCFAttributionABCPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install essential packages

In [22]:
!pip install pandas google-api-python-client numpy google-auth attrs

## Configuring Google Analytics access
To configure Google Analytics, we need some credentials. To speed up the process, follow the steps below:

- In your browser, open [https://developers.google.com/oauthplayground/](https://developers.google.com/oauthplayground/) ![Step 1](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-1.png?raw=1)
- Fill the scope with `https://www.googleapis.com/auth/analytics.readonly` and press "Authorize APIs" ![Step 2](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-2.png?raw=1)
- Enter your credentials and allow the "Google OAuth 2.0 Playground" to access the Google Analytics data
- After entering the credentials, the browser will redirect to a page similar to ![Step 3](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-3.png?raw=1)
- Press "Exchange authorization code for tokens". The tokens will be generated ![Step 4](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-4.png?raw=1)
- Wait until the Step 3 (Configure request to API) appears ![Step 5](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-3.png?raw=1)
- Go back to Step 2 (Exchange authorization code for tokens) and copy the Access Token ![Step 5](https://github.com/stormx-martech/sre-notebooks/blob/main/imgs/Auth-Step-5.png?raw=1)
- Paste the access token in the field below, replacing the `INSERT TOKEN HERE` with the token. Remember to keep the quotes!

# Configuration
We need some information from you to generate the report.

In [23]:
access_token = """
ya29.A0ARrdaM_fDudQmWyQzC-79ndpNJSNBYeAvOBLS47yEFToJBztXzvrRuaWe_d53SrJfSiwYFvVxTtkDZSVT_iHiXA90vOJjh7XwKYjg_CLBksEvBpPdXGsNhVSgKtFc-Ad3KCrngjL3pGmD6G5XMLrFxMRaq-V
""".strip()

print("Checking token...")

import google.oauth2.credentials
import googleapiclient.discovery
    
credentials = google.oauth2.credentials.Credentials(access_token)
ga = googleapiclient.discovery.build("analytics", "v3", credentials=credentials, cache_discovery=False)

try:
    accounts = ga.management().accounts().list().execute()
    print(f"✅ Thank you, {accounts['username']}! The access was granted, and you can continue exploring this notebook.")
except:
    print("❌ There's a problem with the token. 😭")


Checking token...
✅ Thank you, yuri.farias@stormx.com.br! The access was granted, and you can continue exploring this notebook.


## Configuring the data extraction
Please fill the fields below, keeping all the quotes.

In [24]:
# the GA view, e.g: "175359674"
view_id = "175409556" 

# the goals to extract, separed by commas, e.g.: [4, 9]
goals = [18]

# The period, in YYYY-MM-DD format, e.g.: "2021-05-01"
start = "2022-04-18"
end = "2022-04-24"

use_source_medium_path = True
use_campaign_path = True

# The shapley size. 
shapley_size = 1

# The file containing the grouping rules. It's a CSV containg two rows, the search_term and destination. 
grouping_file = "grouping_pf.csv"


# The result will be written at the following locaiton
output_file = "results.csv"


#
# Change the data below only if you know what you're doing!
#
shapley_order = True
conversion_value = "conversions" 
channels_colname = "path" 
last_click_non_but_not_this_channel = "DIRECT"
time_decay_decay_over_time = 0.5 
time_decay_frequency = 128
shapley_values_col = "conv_rate"
group_by_channels_models = True

# ------------------------------------------ #
#   DO NOT CHANGE ANYTHING BELOW THIS LINE   # 
# ------------------------------------------ #
import datetime
try:
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d").date()
    end_date = datetime.datetime.strptime(end, "%Y-%m-%d").date()
except:
    start_date = None
    end_date = None

print("Checking the information...")
print("✅ view_id" if type(view_id) == str and view_id != "" else  "❌ view_id is not properly configured.")
print("✅ goals" if len(goals) > 0 else  "❌ goal is not properly configured.")

print("✅ start" if start_date is not None else  "❌ start is not a valid date.")
print("✅ end" if end_date is not None else  "❌ end is not a valid date.")

print("✅ start/end" if start_date <= end_date is not None else  "❌ the end date happens before the start date.")

print("✅ use_source_medium_path" if type(use_source_medium_path) == bool else  "❌ use_source_medium_path should be True or False, without quotes.")
print("✅ use_campaign_path" if type(use_campaign_path) == bool else  "❌ use_campaign_path should be True or False, without quotes.")

print("✅ shapley_size" if shapley_size >= 1 else  "❌ shapley_size must be greater or equal to 1.")

print("You can proceed only if you get all green marks ✅!")

Checking the information...
✅ view_id
✅ goals
✅ start
✅ end
✅ start/end
✅ use_source_medium_path
✅ use_campaign_path
✅ shapley_size
You can proceed only if you get all green marks ✅!


# Executing the attribution

In [25]:
from datetime import date, datetime
import csv
import time
import sys
import pandas as pd
from time import sleep
from google.oauth2.credentials import Credentials
import googleapiclient.discovery
from google.auth.transport.requests import Request
import datetime
from typing import List, Optional
from enum import Enum
from attr import asdict, attrib, attrs

# adding the current path
sys.path.append('./')

# Same as https://github.com/DP6/Marketing-Attribution-Models, except better error handling and no gui dependencies
from MAM import MAM

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Defining some methods (no configuration)

In [27]:
class PathType(Enum):
    CAMPAIGN = "mcf:campaignPath"
    SOURCE_MEDIUM = "mcf:sourceMediumPath"

In [28]:
def download_mcf_report(
    ga: googleapiclient.discovery.Resource,
    start_date: datetime.date,
    end_date: datetime.date,
    view_id: str,
    goals: List[int],
    path_types: List[PathType],
):
    """Load MCF report."""
    index = 1
    max_results = 10000  # this is limited by google
    rows = []
    conv_filter = ",".join([f"mcf:conversionGoalNumber=={goal:03d}" for goal in goals])

    # parâmetros padrão para o relatório
    default_args = {
        "ids": f"ga:{view_id}",
        "start_date": start_date.strftime("%Y-%m-%d"),
        "end_date": end_date.strftime("%Y-%m-%d"),
        "filters": conv_filter,
        "sort": "mcf:totalConversions",
        "max_results": max_results,
        "dimensions": f"{', '.join(i.value for i in path_types)}, mcf:conversionDate, mcf:conversionGoalNumber",
        "metrics": "mcf:totalConversions",
    }
    print(default_args['dimensions'])

    mcf = ga.data().mcf()

    while index:
        pagination_args = {"start_index": index}

        # merge arguments
        args = {**default_args, **pagination_args}

        sleep_time = 5
        need_mcf = True
        while need_mcf:  # retry loop
            try:
                report = mcf.get(**args).execute()
                need_mcf = False
            except ValueError:
                raise

            except TypeError:
                raise

            except Exception as e:
                if sleep_time > 60 * 20:
                    print(f"Too many errors.")
                    return []
                else:
                    print(f"Throttling request for {sleep_time}")
                    print(e)
                    sleep(sleep_time)
                    sleep_time = sleep_time * 2.5

        # salvando as linhas
        rows.extend(report.get("rows", []))

        # continua no loop? (paginação)
        if not report.get("nextLink"):
            index = 0
        else:
            index += max_results
    report_dict = {
        'rows': rows,
        'column_headers': report.get('columnHeaders', []) 
    }
    return report_dict


In [29]:
def report_row_to_df_row(row):
    # Source Medium path list
    path_list = [step["nodeValue"] for step in row[0]["conversionPathValue"]]

    # Campaign path list
    campaign_path_list = [step["nodeValue"] for step in row[1]["conversionPathValue"]]

    return dict(
        path=path_list,
        campaign_path=campaign_path_list,
        conversions=int(row[-1]["primitiveValue"]),
    )

In [30]:
# execute the report
path_types = []
if use_source_medium_path:
    path_types.append(PathType.SOURCE_MEDIUM)

if use_campaign_path:
    path_types.append(PathType.CAMPAIGN)

report_dict = download_mcf_report(
    start_date = start_date,   
    end_date = end_date,       
    goals = goals,             
    view_id = view_id,         
    path_types = path_types, 
    ga=ga)

report = report_dict['rows']
column_headers = report_dict['column_headers']


mcf:sourceMediumPath, mcf:campaignPath, mcf:conversionDate, mcf:conversionGoalNumber
Throttling request for 5
The read operation timed out


In [31]:
# convert to a dataframe
print(column_headers)
all_data = pd.DataFrame.from_records([report_row_to_df_row(row) for row in report])
print(len(all_data)) # 26875
print(all_data.columns)
print(all_data['campaign_path'])

[{'name': 'mcf:sourceMediumPath', 'columnType': 'DIMENSION', 'dataType': 'MCF_SEQUENCE'}, {'name': 'mcf:campaignPath', 'columnType': 'DIMENSION', 'dataType': 'MCF_SEQUENCE'}, {'name': 'mcf:conversionDate', 'columnType': 'DIMENSION', 'dataType': 'STRING'}, {'name': 'mcf:conversionGoalNumber', 'columnType': 'DIMENSION', 'dataType': 'STRING'}, {'name': 'mcf:totalConversions', 'columnType': 'METRIC', 'dataType': 'INTEGER'}]
26870
Index(['path', 'campaign_path', 'conversions'], dtype='object')
0                                          [pecaseucartao]
1        [pecaseucartao, usuarios_inativos_2_meses_0209...
2        [PERFORMANCE|ABERTURA-DE-CONTA-PF_2021, PERFOR...
3        [PERFORMANCE|ABERTURA-DE-CONTA-PF_2021, Perfor...
4        [PERFORMANCE|ABERTURA-DE-CONTA-PF_2021, PERFOR...
                               ...                        
26865            [cross-sell-app-santander-botao-home-ncc]
26866            [cross-sell-app-santander-botao-home-ncc]
26867            [cross-sell-app-s

## Data Cleanup (no configuration)

In [32]:
try:
    grouping_csv = grouping_file
    # this file is a simple csv with two columns: src and group_name
    with open(grouping_csv) as f:
        path_correlation = csv.DictReader(f).reader
        path_correlation = list(path_correlation)
except:
    print("Unable to load file.")
    path_correlation = [] # you can insert the correlation here, like [["src", "dst"], ..., ["src", "dst"]]

path_correlation = [[x[0].casefold(), x[1]] for x in path_correlation]
path_correlation = dict(path_correlation)
print(path_correlation)

def change_path_string_df(row, path_column, campaign_path_column, raw_output_list=None):
    new_path_items = []

    for source_medium_path, campaign_path in zip(row[path_column], row[campaign_path_column]):
        source_medium_correlated_path = path_correlation.get(source_medium_path.casefold(), source_medium_path)
        raw_output_item_dict = dict()

        raw_output_item_dict['source_medium_item'] = source_medium_path
        raw_output_item_dict['campaigh_item'] = campaign_path
        pf_strings = [("|", "conta-corrente"), ("PERFORMANCE|ABERTURA-DE-CONTA-PF")]
        not_pf_strings = ["pj",]

        contains_pf_strings = any(all(pf_string.lower() in campaign_path.lower() for pf_string in pf_string_rule) for pf_string_rule in pf_strings)
        do_not_contains_other_strings = all(other_string not in campaign_path for other_string in not_pf_strings)
        
        is_path_pf_related = contains_pf_strings and do_not_contains_other_strings
        new_path_item = source_medium_correlated_path

        new_path_item += ' PF' if is_path_pf_related else ' OUTROS'
        raw_output_item_dict['classification'] = new_path_item

        new_path_items.append(new_path_item)
        raw_output_list.append(raw_output_item_dict)

    return " > ".join(new_path_items)


{'source_medium': 'classification', '(direct) / (none)': 'DIRECT', '(not set) / (not set)': 'OUTROS', '(not set) / cpc': 'OUTROS', '[gmb][001-0002] / [website]': 'OUTROS', '[gmb][001-0008] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0010] / [website]': 'OUTROS', '[gmb][001-0013] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0013] / [website]': 'OUTROS', '[gmb][001-0019] / [website]': 'OUTROS', '[gmb][001-0023] / [website]': 'OUTROS', '[gmb][001-0035] / [website]': 'OUTROS', '[gmb][001-0040] / [website]': 'OUTROS', '[gmb][001-0055] / [website]': 'OUTROS', '[gmb][001-0056] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0057] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0058] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0060] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0062] / [website]': 'OUTROS', '[gmb][001-0065] / [website]': 'OUTROS', '[gmb][001-0080] / [website]': 'OUTROS', '[gmb][001-0084] / [produtos][abra-sua-conta]': 'OUTROS', '[gmb][001-0

In [33]:
# apply transformations
all_data_path_changed = all_data.copy().dropna()

# probably you'll need to cleanup a bit more using regular expressions.
SOURCE_MEDIUM_COL = 'path'
CAMPAIGN_PATH_COL = 'campaign_path'
print(all_data_path_changed)
all_data_path_changed.to_csv('raw_output.csv', index=None)
raw_output_list = []
all_data_path_changed["path"] = all_data_path_changed.apply(lambda r: change_path_string_df(r, SOURCE_MEDIUM_COL, CAMPAIGN_PATH_COL, raw_output_list), axis=1)

raw_output_df = pd.DataFrame(raw_output_list)
raw_output_df.to_csv('raw_output.csv', index=None)
#map(change_path_string)
#print(all_data_path_changed['path'])

# Campaign path column regex cleanup
# contem "|" E contem "conta-corrente" E não contem "pj"
# Tudo que cair dentro desta regra, deve ser classificado como "PF", e o restante classificado como "OUTROS".
#print(all_data_path_changed['campaign_path'])
all_data_path_changed = all_data_path_changed.groupby(by=["path"]).sum()

                                                    path  \
0                                            [b2g / sms]   
1      [b2g / sms, insider / web_push, (direct) / (no...   
2      [dbm / cpm, dbm / cpm, dbm / cpm, dbm / cpm, d...   
3      [dbm / cpm, dbm / cpm, dbm / cpm, dbm / cpm, d...   
4      [dbm / cpm, dbm / cpm, dbm / cpm, dbm / cpm, d...   
...                                                  ...   
26865        [cross-sell-app-santander / botao-home-ncc]   
26866        [cross-sell-app-santander / botao-home-ncc]   
26867        [cross-sell-app-santander / botao-home-ncc]   
26868        [cross-sell-app-santander / botao-home-ncc]   
26869        [cross-sell-app-santander / botao-home-ncc]   

                                           campaign_path  conversions  
0                                        [pecaseucartao]            1  
1      [pecaseucartao, usuarios_inativos_2_meses_0209...            1  
2      [PERFORMANCE|ABERTURA-DE-CONTA-PF_2021, PERFOR...       

In [34]:
df = all_data_path_changed.copy().dropna()

## Running the attribution (no configuration)

In [35]:
attributions = MAM.MAM(
    df, conversion_value=conversion_value, channels_colname=channels_colname
)

# don't worry about "A value is trying to be set on a copy of a slice from a DataFrame." error

/content/MAM/MAM.py:370: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: self.sep.join(x)


In [36]:
print("attribution last click...")
attributions.attribution_last_click(
    group_by_channels_models=group_by_channels_models
)

print("attribution last click non direct...")
attributions.attribution_last_click_non(
    but_not_this_channel=last_click_non_but_not_this_channel
)

print("attribution first click...")
attributions.attribution_first_click(
    group_by_channels_models=group_by_channels_models
)

print("attribution linear...")
attributions.attribution_linear(
    group_by_channels_models=group_by_channels_models
)

print("attribution position based...")
attributions.attribution_position_based(
    group_by_channels_models=group_by_channels_models
)

print("attribution time decay...")
attributions.attribution_time_decay(
    decay_over_time=time_decay_decay_over_time,
    frequency=time_decay_frequency,
    group_by_channels_models=group_by_channels_models,
)

print("Done")

attribution last click...
attribution last click non direct...
attribution first click...
attribution linear...
attribution position based...
attribution time decay...
Done


In [37]:
print("attribution shapley...")
start = time.time()
attributions.attribution_shapley(
    size=shapley_size,
    order=shapley_order,
    group_by_channels_models=group_by_channels_models,
    values_col=shapley_values_col,
)

output = attributions.group_by_channels_models
end = time.time()

time_elapsed = end - start
print(f"Done! Time elapsed: {time_elapsed:03.6}s for shapley size = {shapley_size}")

attribution shapley...
Done! Time elapsed: 0.388s for shapley size = 1


In [38]:
try:    
    print("attribution markov... Don't worry about singular matrix errors.")
    attribution_markov = attributions.attribution_markov(
        transition_to_same_state=True
    )
except:
    print("attribution markov failed :(...")
    attribution_markov = None
    pass

print("Done!")


attribution markov... Don't worry about singular matrix errors.
0      34
1      15
2      13
3      16
4      30
       ..
778    28
779    19
780    37
781     8
782     4
Name: dest, Length: 783, dtype: int64
Done!


In [39]:
if attribution_markov is not None:
    removal_effect = (
        attribution_markov[3]
        .reset_index()
        .rename(columns={"index": "channels"})
    )
    output = pd.merge(output, removal_effect, how="left", on="channels")

    output = pd.merge(
        output, attribution_markov[1], how="left", on="channels"
    ).rename(
        columns={
            "attribution_markov_same_state_algorithmic": "markov",
        }
    )

output = output.rename(
    columns={
        "attribution_last_click_heuristic": "last_click",
        "attribution_last_click_non_DIRECT_heuristic": "last_click_non_direct",
        f"attribution_time_decay{time_decay_decay_over_time}_freq{time_decay_frequency}_heuristic": "time_decay",
        "attribution_position_based_0.4_0.2_0.4_heuristic": "position_based",
        "attribution_first_click_heuristic": "first_click",
        "attribution_linear_heuristic": "linear",
        "channels": "source",
    }
)

shapley_columns = [
    f"attribution_shapley_size{shapley_size}_conv_rate_algorithmic",
    f"attribution_shapley_size{shapley_size}_conv_rate_order_algorithmic",
]

for shapley_column in shapley_columns:
    if shapley_column in output.columns:
        output = output.rename(columns={shapley_column: "shapley"})
        break



## Showing the results (no configuration)

In [40]:
output

,source,last_click,last_click_non_direct,first_click,linear,position_based,time_decay,shapley,removal_effect,markov
0,ACELERA OUTROS,1.0,1.0,31.0,8.59,13.958326,7.505218,1.0,0.000764,19.208352
1,ACELERA PF,2050.0,2050.0,1315.0,1512.93,1646.482722,1536.592067,2364.0,0.068731,1729.140707
2,AFILIADO OUTROS,547.0,547.0,487.0,446.08,501.695738,446.173827,595.0,0.028543,718.085467
3,AFILIADO PF,8398.0,8398.0,6444.0,6494.17,7145.687562,6535.155013,8371.0,0.217735,5477.780080
4,AFILIO PF,499.0,499.0,401.0,403.18,435.803631,406.599293,500.0,0.014787,372.016611
5,COMUNICACAO DIRIGIDA OUTROS,3880.0,3880.0,2354.0,2605.95,2970.447463,2642.307730,3874.0,0.110590,2782.225678
6,DFA OUTROS,141.0,141.0,5154.0,2942.66,2733.245071,2756.563080,350.0,0.201429,5067.556142
7,DFA PF,1660.0,1660.0,8426.0,8877.80,6087.851932,8917.210952,2170.0,0.364387,9167.269977
8,DIRECT OUTROS,9596.0,9596.0,2139.0,5890.96,5947.723422,5998.088656,9328.0,0.256557,6454.462791
9,EASYCREDITO PF,95.0,95.0,63.0,70.66,77.116253,71.460784,95.0,0.002589,65.124147


# Writing the output

In [41]:
output.to_csv(output_file, index=False,float_format="%1.6f",sep=';')
print ('Operation complete!')

Operation complete!
